Load modules

In [1]:
import numpy as np
import pandas as pd

# Our modules
from utils.minmax_hart_norm import minmax_hart_convert, minmax_hart_revert

csv file load

In [2]:
csv_file = pd.read_csv('train_y_wh.csv')
csv_arr = csv_file.to_numpy()[0, :-2]

print(csv_arr.shape)

(68,)


In [3]:
width = csv_file.to_numpy()[0, -2, np.newaxis]
height = csv_file.to_numpy()[0, -1, np.newaxis]

print(width.shape)
print(height.shape)

(1,)
(1,)


Convert

In [4]:
noise_arr = np.load('utils/landmark_std.npy')

In [17]:
converted_arr, mat_calc, df_y = minmax_hart_convert(csv_arr, width, height, noise_arr=None,\
                                                    parm=None, noise=False)
converted_arr_2, mat_calc_2, df_y_2 = minmax_hart_convert(csv_arr, width, height, noise_arr=noise_arr,\
                                                          parm=None, noise=True, alpha=10000)

alpha : 10000


In [19]:
converted_arr_2-converted_arr

array([ 1.26780769e-01,  1.26978282e+00, -2.55394059e-01,  3.16235953e-01,
       -3.04797830e-01, -1.36499685e-01, -1.31394246e-01, -1.03888090e-02,
       -2.76319116e-01, -3.71431474e-01, -1.76336884e-01, -1.83474117e-01,
       -5.97683700e-02, -1.92665528e-03,  9.47791312e-02, -1.39965363e-01,
       -1.47098347e-01, -2.76945285e-01,  2.00443263e-01, -1.43926710e-01,
        4.04007800e-02, -2.03467777e-01,  6.87310384e-02, -1.92097909e-01,
        1.34792120e-01, -1.29124412e-01,  1.39545011e-02, -2.89182846e-01,
       -1.15948833e-01, -1.49970248e-01, -3.87319359e-02, -3.06933489e-01,
        1.64961011e-03, -3.16535753e-01, -1.54831628e-02, -2.05962628e-01,
       -3.55624127e-02, -1.21046194e-01, -1.44803105e-01, -1.23340825e-01,
       -7.79019651e-02, -1.56815538e-01, -1.33355568e-01, -2.92045995e-01,
       -2.81147357e-02, -3.80553226e-02, -6.25944908e-03,  1.48595202e+00,
        7.28854174e-05,  6.16045500e-01,  1.82889723e-01, -4.19409016e-02,
        1.22798451e-01, -

Revert

In [9]:
reverted_arr = minmax_hart_revert(converted_arr, mat_calc, width, height)
reverted_arr_2 = minmax_hart_revert(converted_arr_2, mat_calc_2, width, height)

In [10]:
reverted_arr

array([1269.,  774.,  817.,  857.,  656., 1002.,  916., 1020.,  639.,
       1159.,  711., 1090., 1175.,  990., 1283., 1140.,  963., 1149.,
       1267., 1195., 1293., 1271., 1232., 1204., 1283., 1360., 1273.,
       1368., 1216., 1483., 1263., 1444., 1228., 1495., 1234., 1607.,
       1212., 1648., 1183., 1656.,  737., 1401., 1059., 1312., 1065.,
       1358., 1328.,  664., 1316.,  841., 1438., 1104., 1336., 1206.,
       1354., 1289., 1371., 1338., 1310., 1395., 1364., 1476., 1307.,
       1517., 1299., 1617., 1257., 1690.])

---
확인

In [11]:
print(np.max((reverted_arr - csv_arr)))
print(np.min((reverted_arr - csv_arr)))

2.2737367544323206e-13
0.0


In [12]:
print(np.max((reverted_arr)))
print(np.min((reverted_arr)))
print(np.max((csv_arr)))
print(np.min((csv_arr)))

1690.0000000000002
639.0
1690.0
639.0


In [10]:
reverted_arr_2 = np.concatenate((reverted_arr, width, height), axis=0)
df_y2 = np.concatenate((df_y, width, height), axis=0)

print(reverted_arr_2.shape)

(70,)


In [11]:
reverted_arr_2

array([1269.,  774.,  817.,  857.,  656., 1002.,  916., 1020.,  639.,
       1159.,  711., 1090., 1175.,  990., 1283., 1140.,  963., 1149.,
       1267., 1195., 1293., 1271., 1232., 1204., 1283., 1360., 1273.,
       1368., 1216., 1483., 1263., 1444., 1228., 1495., 1234., 1607.,
       1212., 1648., 1183., 1656.,  737., 1401., 1059., 1312., 1065.,
       1358., 1328.,  664., 1316.,  841., 1438., 1104., 1336., 1206.,
       1354., 1289., 1371., 1338., 1310., 1395., 1364., 1476., 1307.,
       1517., 1299., 1617., 1257., 1690., 2010., 1670.])

---
한번에 변환

In [12]:
x_arr       = []
y_arr       = []
mat_array   = []


for i in range(len(csv_file)):
# for i in range(5):
    csv_arr = csv_file.to_numpy()[i, :-2]
    converted_arr, mat_calc, df_y = minmax_hart_convert(csv_arr, width, height, parm = None)
    
    x_arr.append(converted_arr)
    y_arr.append(df_y)
    mat_array.append(mat_calc)
    
a = pd.DataFrame(np.array(x_arr))
b = pd.DataFrame(np.array(y_arr))

a.columns = csv_file.columns[:-2]
b.columns = csv_file.columns[:-2]

a.to_csv('./train_x_wh_hart.csv', index=False)
b.to_csv('./train_y_wh_hart.csv', index=False)

np.save('./mat_calc.npy', mat_calc)